In [ ]:
import pandas as pd
import numpy as np

data = [
    {'game_id': '2025-09-07_BALvsPIT', 'p_xgb': 0.55, 'p_book': 0.52, 'p_polymarket': 0.50},
    {'game_id': '2025-09-07_NYGvsDAL', 'p_xgb': 0.60, 'p_book': 0.58, 'p_polymarket': 0.62},
]
signals_df = pd.DataFrame(data)

#accounting for vig
def normalize_book_odds(p_book_raw):
    p_book_home, p_book_away = p_book_raw, 1 - p_book_raw  
    total = p_book_home + p_book_away
    return p_book_home / total

class BMAWeights:
    def __init__(self, num_models, alpha=1.0, prior=None):
        self.alpha = alpha
        self.log_weights = np.log(prior) if prior is not None else np.log(np.ones(num_models) / num_models)
    
    def update(self, y_true, preds):
        self.log_weights = self.alpha * self.log_weights + np.log(preds)
        max_log = np.max(self.log_weights)
        self.log_weights -= max_log
        weights = np.exp(self.log_weights)
        self.weights = weights / weights.sum()
        return self.weights

def compute_fair_value(row, weights):
    model_probs = np.array([row['p_xgb'], row['p_book'], row['p_polymarket']])
    return np.dot(weights, model_probs)

bma = BMAWeights(num_models=3, alpha=0.98)

test = [
    (1, [0.55, 0.52, 0.50]),  
    (0, [0.60, 0.58, 0.62]),  
    (1, [0.53, 0.50, 0.48]),
]

for y_true, preds in test:
    model_preds = np.array(preds) if y_true == 1 else 1 - np.array(preds)
    weights = bma.update(y_true, model_preds)

signals_df['fair_p'] = signals_df.apply(lambda r: compute_fair_value(r, bma.weights), axis=1)
signals_df['fair_odds'] = 1 / signals_df['fair_p']

signals_df


,game_id,p_xgb,p_book,p_polymarket,fair_p,fair_odds
0,2025-09-07_BALvsPIT,0.55,0.52,0.50,0.525244,1.903879
1,2025-09-07_NYGvsDAL,0.60,0.58,0.62,0.598885,1.669768


In [3]:
import os
os.environ.pop('MPLBACKEND', None)
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, roc_auc_score, brier_score_loss
from sklearn.calibration import calibration_curve
import xgboost as xgb
import matplotlib.pyplot as plt
import pickle

np.random.seed(42)
n_samples = 500

df = pd.DataFrame({
    'team_stat_diff': np.random.normal(0, 1, n_samples),
    'rest_day_diff': np.random.randint(-1, 3, n_samples),
    'home_field_advantage': np.random.binomial(1, 0.6, n_samples),
})
logit = 0.5 * df['team_stat_diff'] + 0.3 * df['rest_day_diff'] + 0.8 * df['home_field_advantage']
prob = 1 / (1 + np.exp(-logit)) #sigmoid
df['home_win'] = np.random.binomial(1, prob)

TARGET = 'home_win'
feature_cols = [c for c in df.columns if c != TARGET] #removes home_win for inputs
X = df[feature_cols] 
y = df[TARGET]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=42
)#20% saved for validation, rest training to prevent overfitting

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val,label=y_val)


params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 4,
    'eta': 0.1, #learning rate
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'lambda': 1, #L2 regularization
    'seed': 42
}

model = xgb.train(
    params,
    dtrain,
    num_boost_round=100,
    evals=[(dtrain, 'train'), (dval, 'validation')],
    early_stopping_rounds=10,
    verbose_eval=False
)

y_pred_val = model.predict(dval)
print("Validation Log-loss:", log_loss(y_val, y_pred_val)) #ideally <0.5 for probability estimate accuracy
print("ROC AUC:", roc_auc_score(y_val, y_pred_val))
print("Brier Score:", brier_score_loss(y_val, y_pred_val))

prob_true, prob_pred = calibration_curve(y_val, y_pred_val, n_bins=10)
plt.figure()
plt.plot(prob_pred, prob_true, marker='o', linewidth=1)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel('Predicted Probability')
plt.ylabel('True Probability')
plt.title('Calibration Curve on Pseudo Data')
plt.savefig('calibration_curve.png')
with open('xgb_pseudo_model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Pseudo-model saved to xgb_pseudo_model.pkl")

Validation Log-loss: 0.6169348241026441
ROC AUC: 0.6854838709677419
Brier Score: 0.2140811476917003
Pseudo-model saved to xgb_pseudo_model.pkl
